<a href="https://colab.research.google.com/github/danielelbrecht/mirna/blob/master/mirna_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import numpy as np


from keras.models import Model
from keras.layers import Input, LSTM, TimeDistributed, Dropout, Dense, Permute, Flatten, Multiply, RepeatVector, Activation, Masking, Bidirectional
from keras import regularizers, optimizers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.wrappers import Wrapper
from keras.engine.topology import InputSpec
from keras import backend as K

In [0]:
# Load data sets

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

pos_file_obj = drive.CreateFile({'id': '1vl-qE0U5W6ll3JH41QqDajyx6oAwC3C0'})                       
pos_file_obj.GetContentFile('input.txt')

neg_file_obj = drive.CreateFile({'id': '1Rnh8RHUsmCGmiCZobu3g7ezeUJQq0CH-'})                       
neg_file_obj.GetContentFile('negatives.txt')

In [0]:
pos_file_obj
pos_content = pos_file_obj.GetContentString()
neg_content = neg_file_obj.GetContentString()
pos_file = []
neg_file = []
temp = []

for x in pos_content:
  if x == '\n':
    pos_file.append(temp)
    temp = []
  else: 
    temp.append(x)
    
for x in neg_content:
  if x == '\n':
    neg_file.append(temp)
    temp = []
  else: 
    temp.append(x)
    
    
def read_line(line):

    array = []

    for entry in line:
        if entry == '0':
            array.append(np.int32(0))
        if entry == '1':
            array.append(np.int32(1))
        if len(array) == 16:
          break

    return np.asarray(array)
  
def process_data(pos_file, neg_file):

    data = []
    is_example = 0
    pos_examples = 0
    neg_examples = 0

    for line in pos_file: # Iterate over file

        if (line[0] == '0' or line[0] == '1') and is_example == 0:  # When new sequence is encountered, initialize new example
            example = []
            is_example = 1
            example.append(read_line(line))

        if (line[0] == '0' or line[0] == '1') and is_example == 1:  # During sequence
            example.append(read_line(line))

        if line[0] != '0' and line[0] != '1' and is_example == 1:  # When sequence terminates
            is_example = 0
            data.append(example)
            pos_examples = pos_examples + 1

    for line in neg_file: # Iterate over file

        if (line[0] == '0' or line[0] == '1') and is_example == 0:  # When new sequence is encountered, initialize new example
            example = []
            is_example = 1
            example.append(read_line(line))

        if (line[0] == '0' or line[0] == '1') and is_example == 1:  # During sequence
            example.append(read_line(line))

        if line[0] != '0' and line[0] != '1' and is_example == 1:  # When sequence terminates
            is_example = 0
            data.append(example)
            neg_examples = neg_examples + 1

    return np.asarray(data), pos_examples, neg_examples

In [0]:
# Process the data and generate training labels

full_data, num_pos, num_neg = process_data(pos_file, neg_file)

# Generate labels
pos_labels = np.ones(num_pos)
neg_labels = np.zeros(num_neg)
data_labels = np.concatenate((pos_labels, neg_labels))

binary_labels = np.zeros([len(data_labels), 2])

for i in range(len(data_labels)):
    if data_labels[i] == 1:
        binary_labels[i][1] = 1
    else:
        binary_labels[i][0] = 1


# Get mask length
mask_length = 0
for i in range(len(full_data)):
    if len(full_data[i]) > mask_length:
        mask_length = len(full_data[i])


In [6]:
# Pad data
data_padded = pad_sequences(full_data, maxlen=mask_length, dtype='object', padding='post', truncating='post', value=0)

# Shuffle data and get training and validation sets
indices = np.random.permutation(35267)
shuffled_data = data_padded[indices]
shuffled_labels = binary_labels[indices]

print(data_padded.shape)
print(shuffled_data.shape)


(35267, 142, 16)
(35267, 142, 16)


In [0]:
# Define hyper parameters
LSTM_units = 32
fully_connected_layer_units = 32
output_size = 2
learning_rate = 0.01



In [44]:
# Functional API model

# Input layer
inputs = Input(shape=(mask_length, 16), name='inputs')


# LSTM Layers
lstm1 = Bidirectional(LSTM(20, return_sequences=True, recurrent_dropout=0.1), merge_mode='concat')(inputs)
lstm2 = Bidirectional(LSTM(10, return_sequences=True, recurrent_dropout=0.1), merge_mode='concat')(lstm1)

#Flatten
flatten = Flatten()(lstm2)


# Fully connected layers
do1 = Dropout(0.1)(flatten)
fc1 = Dense(100, activation='sigmoid')(do1)
do2 = Dropout(0.1)(fc1)
fc2 = Dense(100, activation='sigmoid')(do2)

# Output layer
softmax = Dense(output_size, activation='softmax')(fc2)


# Compile model
model2 = Model(inputs=inputs, outputs=softmax)

model2.compile(optimizer='rmsprop',
               loss='binary_crossentropy',
               metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 142, 16)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 142, 40)           5920      
_________________________________________________________________
bidirectional_5 (Bidirection (None, 142, 20)           4080      
_________________________________________________________________
flatten_11 (Flatten)         (None, 2840)              0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 2840)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 100)               284100    
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
__________

In [0]:
def kfold(k, model, data, labels):
  
  epochs = 10
  length = len(data)
  accuracy = 0
  model.save_weights('initial_weights')
  
  for i in range(k):
    
    model.load_weights('initial_weights')
    
    #Get validation and training splits from data set
    lower_bound = int(i*(length/k))
    upper_bound = int((i+1)*(length/k))

    train_data = np.concatenate((data[0:lower_bound], data[upper_bound:length]))
    val_data = data[lower_bound:upper_bound]
    
    train_labels = np.concatenate((labels[0:lower_bound], labels[upper_bound:length]))
    val_labels = labels[lower_bound:upper_bound]
    
    history = model.fit(train_data, 
                      train_labels, 
                      epochs=epochs,
                      batch_size = 128,
                      validation_data=(val_data, val_labels))
    
    accuracy = accuracy + history.history['acc'][epochs-1]
    

  return accuracy / k


In [45]:
# Run 5 fold cross validation
kfold_acc = kfold(5, model2, shuffled_data, shuffled_labels)
print('5-vold cross validation accuracy is: ', kfold_acc)

Train on 28214 samples, validate on 7053 samples
Epoch 1/10
28214/28214 [==============================] - 214s 8ms/step - loss: 0.4147 - acc: 0.8146 - val_loss: 0.3796 - val_acc: 0.8266
Epoch 2/10
28214/28214 [==============================] - 209s 7ms/step - loss: 0.3510 - acc: 0.8443 - val_loss: 0.3605 - val_acc: 0.8411
Epoch 3/10
28214/28214 [==============================] - 209s 7ms/step - loss: 0.3331 - acc: 0.8556 - val_loss: 0.3170 - val_acc: 0.8606
Epoch 4/10
28214/28214 [==============================] - 206s 7ms/step - loss: 0.3214 - acc: 0.8606 - val_loss: 0.3234 - val_acc: 0.8579
Epoch 5/10
28214/28214 [==============================] - 210s 7ms/step - loss: 0.3126 - acc: 0.8661 - val_loss: 0.3034 - val_acc: 0.8679
Epoch 6/10
28214/28214 [==============================] - 209s 7ms/step - loss: 0.3061 - acc: 0.8690 - val_loss: 0.2959 - val_acc: 0.8737
Epoch 7/10
28214/28214 [==============================] - 206s 7ms/step - loss: 0.2984 - acc: 0.8724 - val_loss: 0.3098 - v

In [0]:
def perfeval(predictions, Y_test, verbose=0):
 class_label = np.uint8(np.argmax(predictions,axis=1))
 R = np.asarray(np.uint8([sublist[1] for sublist in Y_test]))
 CM = metrics.confusion_matrix(R, class_label, labels=None)
 
 CM = np.double(CM)
 acc = (CM[0][0]+CM[1][1])/(CM[0][0]+CM[0][1]+CM[1][0]+CM[1][1])
 se = (CM[0][0])/(CM[0][0]+CM[0][1])
 sp = (CM[1][1])/(CM[1][0]+CM[1][1])
 f1 = (2*CM[0][0])/(2*CM[0][0]+CM[0][1]+CM[1][0])
 ppv = (CM[0][0])/(CM[0][0]+CM[1][0])
 mcc = (CM[0][0]*CM[1][1]-CM[0][1]*CM[1][0])/np.sqrt((CM[0][0]+CM[0][1])*(CM[0][0]+CM[1][0])*(CM[0][1]+CM[1][1])*(CM[1][0]+CM[1][1]))
 gmean = np.sqrt(se*sp)
 auroc = metrics.roc_auc_score(Y_test[:,0],predictions[:,0])
 aupr = metrics.average_precision_score(Y_test[:,0],predictions[:,0],average="micro")
  
 if verbose == 1:
  print("SE:","{:.3f}".format(se),"SP:","{:.3f}".format(sp),"F-Score:","{:.3f}".format(f1), "PPV:","{:.3f}".format(ppv),"gmean:","{:.3f}".format(gmean),"AUROC:","{:.3f}".format(auroc), "AUPR:","{:.3f}".format(aupr))
 
 return [se,sp,f1,ppv,gmean,auroc,aupr,CM]

In [48]:
from sklearn import metrics

# Get advanced metrics
preds = model2.predict(shuffled_data[0:5000])
met = perfeval(preds, shuffled_labels[0:5000], 1)

SE: 0.876 SP: 0.901 F-Score: 0.883 PPV: 0.891 gmean: 0.888 AUROC: 0.959 AUPR: 0.958
